In [1]:
!env | grep VIRTUAL_ENV

VIRTUAL_ENV=/media/bigalex95pc/Extra Space/git/pix2pix_TecoGAN/venv


In [5]:
### IMPORTING NECESSARY LIRARIES
import cv2
import tensorflow as tf
import numpy as np
import torch
import pyfakewebcam
from threading import Thread

import yaml
from TecoGAN_PyTorch.codes.utils import base_utils
import os.path as osp
from TecoGAN_PyTorch.codes.models import define_model
from imutils.video import FPS

In [ ]:
### PIX2PIX AND TECOGAN CONFIGURATIONS
SIZE = 256
NORM = 127.5

In [ ]:
### TENSORFLOW GPU CONFIGURATIONS
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_memory_growth(gpus[0], True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

In [6]:
### PYTORCH GPU CONFIGURATIONS
device = torch.device('cuda:0')
torch.cuda.set_device(device)
print('Active CUDA Device: GPU', torch.cuda.current_device())

Active CUDA Device: GPU 0


In [7]:
### USE OPENCV WITH MULTITHREADING
class WebcamVideoStream:
    def __init__(self, src=0, device=None):
        # initialize the video camera stream and read the first frame
        # from the stream
        self.stream = cv2.VideoCapture(src, device)
        (self.grabbed, self.frame) = self.stream.read()
        # initialize the variable used to indicate if the thread should
        # be stopped
        self.stopped = False

    def start(self):
        # start the thread to read frames from the video stream
        Thread(target=self.update, args=()).start()
        return self

    def update(self):
        # keep looping infinitely until the thread is stopped
        while True:
            # if the thread indicator variable is set, stop the thread
            if self.stopped:
                return

            # otherwise, read the next frame from the stream
            (self.grabbed, self.frame) = self.stream.read()

    def read(self):
        # return the frame most recently read
        return self.frame

    def stop(self):
        # indicate that the thread should be stopped
        self.stopped = True

In [ ]:
### LOAD PIX2PIX MODEL
generator = tf.saved_model.load("./model/pix2pixTF-TRT")

In [ ]:
### FUNCTION FOR USING PIX2PIX
def pix2pix(img):
    # print(img.shape)
    # print(type(img))
    input_image = tf.cast(img, tf.float32)
    input_image = tf.image.resize(input_image, [SIZE, SIZE],
                                    method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    input_image = (input_image / NORM) - 1
    ext_image = tf.expand_dims(input_image, axis=0)
    prediction = generator(ext_image, training=True)
    pil_image = tf.keras.preprocessing.image.array_to_img(prediction[0])
    return pil_image

# def get_from_model(iq, oq):
#     # print(style.RED + str(oq.maxsize))
#     while not exitFlag:
#         if not iq.empty():
#             image = iq.get()
#             # print(image.shape)
#             # print(type(image))
#             input_image = tf.cast(image, tf.float32)
#             input_image = tf.image.resize(input_image, [SIZE, SIZE],
#                                           method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
#             input_image = (input_image / NORM) - 1
#             # input_image = load_from_video(image)
#             ext_image = tf.expand_dims(input_image, axis=0)
#             prediction = generator(ext_image, training=True)
#             # generated_image = generate_images(generator, ext_image)
#             # pil_image = tf.keras.preprocessing.image.array_to_img(
#             #     generated_image)
#             pil_image = tf.keras.preprocessing.image.array_to_img(
#                 prediction[0])
#             if not oq.full():
#                 oq.put(np.array(pil_image))

In [8]:
### FUNCTION FOR USING TECOGAN
with open(osp.join("./TecoGAN_PyTorch/experiments_BD/TecoGAN/001", "live.yml"), 'r') as f:
    opt = yaml.load(f.read(), Loader=yaml.FullLoader)

# ----------------- general configs ----------------- #
# experiment dir
opt['exp_dir'] = "./TecoGAN-PyTorch/experiments_BD/TecoGAN/001"

# random seed
base_utils.setup_random_seed(opt['manual_seed'])

# logger
base_utils.setup_logger('base')
opt['verbose'] = opt.get('verbose', False)

# device
opt['device'] = 'cuda'

# setup paths
base_utils.setup_paths(opt, mode='test')

# run
opt['is_train'] = False


def live(opt):
    cv2.namedWindow("LR image", cv2.WINDOW_NORMAL)
    cv2.namedWindow("HR image", cv2.WINDOW_NORMAL)
    # cap = cv2.VideoCapture(0)
    cap = WebcamVideoStream(src=0).start()

    # logging
    logger = base_utils.get_logger('base')
    if opt['verbose']:
        logger.info('{} Configurations {}'.format('=' * 20, '=' * 20))
        base_utils.print_options(opt, logger)

    # infer and evaluate performance for each model
    for load_path in opt['model']['generator']['load_path_lst']:
        # setup model index
        model_idx = osp.splitext(osp.split(load_path)[-1])[0]

        # log
        logger.info('=' * 40)
        logger.info('Testing model: {}'.format(model_idx))
        logger.info('=' * 40)

        # create model
        opt['model']['generator']['load_path'] = load_path
        model = define_model(opt)
        fps = FPS().start()
        while True:
            image = cap.read()
            if image.any():
                # print(image.shape)
                image = cv2.resize(image, (128, 128))
                cv2.imshow("LR image", image)
                # cv2.imwrite("LR_image.png", image)
                norm_image = cv2.normalize(
                    image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                tmp_torch = torch.from_numpy(norm_image[None, :, :, :]).cuda()
                hr_image = model.infer_live(tmp_torch)
                cv2.imshow("HR image", hr_image[0])
                # cv2.imwrite("HR _image.png", hr_image[0])
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    cv2.destroyAllWindows()
                    cap.stop()
                    break
                fps.update()
    fps.stop()
    print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
    # logging
    logger.info('Finish testing')
    logger.info('=' * 40)

2021-04-27 03:25:37,648 [INFO]: ==================== Configurations ====================
2021-04-27 03:25:37,652 [INFO]: scale: 4
2021-04-27 03:25:37,657 [INFO]: manual_seed: 0
2021-04-27 03:25:37,661 [INFO]: verbose: True
2021-04-27 03:25:37,664 [INFO]: dataset:
2021-04-27 03:25:37,667 [INFO]:   degradation:
2021-04-27 03:25:37,670 [INFO]:     type: BD
2021-04-27 03:25:37,690 [INFO]:     sigma: 1.5
2021-04-27 03:25:37,697 [INFO]:   test1:
2021-04-27 03:25:37,701 [INFO]:     name: Vid4
2021-04-27 03:25:37,704 [INFO]:     gt_seq_dir: TecoGAN_PyTorch/data/Vid4/GT
2021-04-27 03:25:37,706 [INFO]:     lr_seq_dir: TecoGAN_PyTorch/data/Vid4/Gaussian4xLR
2021-04-27 03:25:37,708 [INFO]:     filter_file: None
2021-04-27 03:25:37,709 [INFO]:     num_workers: 4
2021-04-27 03:25:37,712 [INFO]:     pin_memory: True
2021-04-27 03:25:37,714 [INFO]:   test2:
2021-04-27 03:25:37,715 [INFO]:     name: ToS3
2021-04-27 03:25:37,716 [INFO]:     gt_seq_dir: TecoGAN_PyTorch/data/ToS3/GT
2021-04-27 03:25:37,71

In [9]:
### testing area
live(opt)

2021-04-27 03:26:12,434 [INFO]: ==================== Configurations ====================
2021-04-27 03:26:12,435 [INFO]: scale: 4
2021-04-27 03:26:12,436 [INFO]: manual_seed: 0
2021-04-27 03:26:12,436 [INFO]: verbose: True
2021-04-27 03:26:12,437 [INFO]: dataset:
2021-04-27 03:26:12,438 [INFO]:   degradation:
2021-04-27 03:26:12,439 [INFO]:     type: BD
2021-04-27 03:26:12,440 [INFO]:     sigma: 1.5
2021-04-27 03:26:12,441 [INFO]:   test1:
2021-04-27 03:26:12,442 [INFO]:     name: Vid4
2021-04-27 03:26:12,443 [INFO]:     gt_seq_dir: TecoGAN_PyTorch/data/Vid4/GT
2021-04-27 03:26:12,444 [INFO]:     lr_seq_dir: TecoGAN_PyTorch/data/Vid4/Gaussian4xLR
2021-04-27 03:26:12,445 [INFO]:     filter_file: None
2021-04-27 03:26:12,446 [INFO]:     num_workers: 4
2021-04-27 03:26:12,447 [INFO]:     pin_memory: True
2021-04-27 03:26:12,448 [INFO]:   test2:
2021-04-27 03:26:12,449 [INFO]:     name: ToS3
2021-04-27 03:26:12,449 [INFO]:     gt_seq_dir: TecoGAN_PyTorch/data/ToS3/GT
2021-04-27 03:26:12,45

In [ ]:
### FONCTION FOR DENOISING WITH BILETAREL
def denoising(img):
    img = cv2.bilateralFilter(img,15,80,80)
    return img

In [ ]:
### FUNCTION FOR WEBCAM
camera = pyfakewebcam.FakeWebcam('/dev/video0', SIZE, SIZE)
def send_to_webcam(img):
    camera.schedule_frame(img)

In [ ]:
### FUNCTION FOR SHOWING IMAGES
cv2.namedWindow("test window", cv2.WINDOW_NORMAL)
cv2.destroyAllWindows()
def showImages(img):
    if not tf.is_tensor(img):
        cv2.imshow("test window", img)
    else:
        cv2.imshow("test window", img.numpy())
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     cv2.destroyAllWindows()

In [ ]:
### testing area
while True:
    demo = np.random.rand(256, 256, 3)
    showImages(demo)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break